In [31]:
import os 
os.chdir('/Users/egill/dev/genki/genki-signals')

import time
import numpy as np
from threading import Thread

from genki_signals.buffers import DataBuffer
from genki_signals.signal_sources import WaveSignalSource
import genki_signals.signal_functions as sf
from genki_signals.signal_system import SignalSystem

In [32]:
wave_source = WaveSignalSource(ble_address='7CEF8BE3-F544-99C9-B25A-3C6DD1281491', spectrogram=True)

In [33]:
derived = [
    sf.Concatenate(['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z'], name='concat'),
    sf.Reshape('concat', shape=(6, 16), name='model_input'),
    sf.Inference(
        'genki_signals/models/is_touching_model/model.onnx', 
        'model_input', 
        stateful=True, 
        name='is_touching_inference',
        init_state = np.zeros((1,512))
    )
]

system = SignalSystem(wave_source, derived_signals=derived)
buffer = DataBuffer(maxlen=400)

def run_system():
    system.start()
    while True:
        buffer.extend(system.read())
        time.sleep(1 / 50)
        if not system.is_active:
            return

t = Thread(target=run_system)
t.start()

Connecting to wave at address 7CEF8BE3-F544-99C9-B25A-3C6DD1281491
Connected to Wave


In [34]:
buffer.plot('is_touching_inference', plot_type='histogram', class_names=['touching_still', 'touching_moving', 'in_air_still', 'in_air_moving'])

Figure(axes=[Axis(scale=OrdinalScale()), Axis(label='Probability', orientation='vertical', scale=LinearScale()…

In [35]:
system.stop(); t.join()

Got a cancel message, exiting.


In [36]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!
